<a href="https://colab.research.google.com/github/Supramit-Dutta/MyCalcRepo/blob/main/Session_6_The_Adventure_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order to use this document, first make a copy of it on your drive.

File / Save a copy on drive

If you work with another student, share your document with him/her.

click on the share icon on the top right corner


In [ ]:

# Created by Yannick Meiller in February 2016 (initial development was in Java)
# Adapted in Python by Yannick Meiller in February 2020

# This class is the main class of a simple,text-based, adventure game.
# This game is inspired by the Zorg game, developed in March 2000 by Michael Kolling - Monash University.
# 
# The universe of this game is made of interconnected rooms.
# The player can move through these rooms.
# In this basic version the goal is to reach a particular room.
# This game can be extended  - and it should be to be more interesting !
# 


# IMPORTANT
# In this version, the attributes are not created explicitly private
# However, they shall not be accessed directly (remember the best practice!)
# In each class, there exist a series of methods to change or get the value of these attributes
# (getters and setters)



#_______________________________________________________________________________


# This Game class creates and initialises all the others: it creates all
# rooms, creates the parser and starts the game.  It also evaluates the
# commands that the parser identified.
#

class Game :


   ##
    # This is a constructor.
    # It creates the game and initialises its internal map.
    #
   def __init__(self) :
       self.wantsToQuit = False
       self.currentRoom = None #None is some sort of null object - currentRoom will be defined later
       self.goalRoom = None
       self.playerChannel = Interaction()
       self.dirVocab = Vocabulary(self.playerChannel.getDirections())
       self.commandVocab = Vocabulary(self.playerChannel.getCommands())
       self.createRooms()
       self.parser = Parser(2)  # the parameter 2 here in the call to the constructor of `Parser` means
       # that in this game sentences typed in by the player will contain at most 2 words.



   ##
    # Main play routine. Loops until end of play.
    #
   def play(self) :
       self.welcome()
       # Enter the main command loop. Here we repeatedly read commands and execute them until the
       # game is over.

       self.wantsToQuit = False
       while not(self.currentRoom.isSameRoomAs(self.goalRoom))  and not self.wantsToQuit :
           # ! is the logical operator NOT
           # `currentRoom.isSameRoomAs(goalRoom)` checks whether we have reached the goal. 
           self.parser.getAndAnalyzeSentence()
           if self.parser.lastSentenceLength() > 0 : # Here we know the player has typed in at least 1 word
               if self.commandVocab.isLexiconWord(self.parser.wordAt(1)) : # Here we know the first word
                   # typed in corresponds to a known command
                   self.executeCommand()
               else :
                   self.playerChannel.expressGeneralMisunderstanding()
            
           print(self.currentRoom.getLongDescription())
       
       if not self.wantsToQuit : # This means we are here because the player has won
           self.playerChannel.expressCongratulations()
       
       self.playerChannel.expressGoodBye()
   



   ##
    # Create all the rooms and link their exits together.
    #
   def createRooms(self) :

       # Create the rooms
       mainYard = Room("the central yard of ESCP Europe - Paris campus", self.dirVocab)
       quatter = Room("the Quatter - the student bar", self.dirVocab)
       library = Room("the library of ESCP Europe in Paris", self.dirVocab)
       vitalRoux = Room("Vital Roux - the largest conference room", self.dirVocab)
       cafet = Room("La Cafet' - the place for snacking", self.dirVocab)
       hall2 = Room("Hall of building2", self.dirVocab)

       # Initialise room exits
       mainYard.setExit("West", vitalRoux)
       mainYard.setExit("East", library)
       mainYard.setExit("South", quatter)
       mainYard.setExit("North", hall2)
       quatter.setExit("North", mainYard)
       library.setExit("West", mainYard)
       vitalRoux.setExit("East", mainYard)
       vitalRoux.setExit("North", cafet)
       cafet.setExit("South", vitalRoux)
       cafet.setExit("East", hall2)
       hall2.setExit("West", cafet)
       hall2.setExit("South", mainYard)

       self.currentRoom = cafet  # Start game in the Room cafet
       self.goalRoom = quatter   # The goal is to reach the Room quatter
   

   ##
    # Print out the opening message for the player.
    #
   def welcome(self) :

       self.playerChannel.expressWelcome()
       self.playerChannel.expressVintage()
       self.playerChannel.expressHowToGetHelp()

       self.playerChannel.expressCurrentLocation(self.currentRoom)

       self.playerChannel.expressGoal(self.goalRoom)

   

   ##
    # initial state : a sentence typed in by the player has been obtained and analyzed
    # its first word is a known command
    # final state : the command has been processed.
    # Either the following words made sense with the command, and an action in the game has been executed
    # or the system has said to the player it did not understand
    #
   def executeCommand(self) :

       firstWord = self.parser.wordAt(1)
       if firstWord == "Quit" :
         self.wantsToQuit = True
       elif firstWord == "Help" :
         self.printHelp()
       elif firstWord == "Go" :
         # The command "Go" requires another word to indicate in which direction to go
         if (self.parser.lastSentenceLength() < 2) : # here the player'sentence does not contain a second word
            self.playerChannel.expressMissingDirection()
         else : # Here the first word is Go and there is a second word
            secondWord = self.parser.wordAt(2)
            if self.dirVocab.isLexiconWord(secondWord):  #here we know this second word is a known direction
                if self.currentRoom.isExit(secondWord): #here we know in that direction there is indeed an exit
                    self.currentRoom = self.currentRoom.nextRoom(secondWord) #the player moves in that direction and reaches the next room
                else :
                    self.playerChannel.expressInTheWall()
            else :#here we know the second word does not describe a known direction
                self.playerChannel.expressDirectionMisunderstanding()
            
       else:
         self.playerChannel.expressGeneralMisunderstanding()


   ##
    # Print out some help information.
    # Here we print a strange and sad message, as welle as the list of possible command words and
    # the list of possible directions.
    # Is this really helpful ? ;-)
    #
   def printHelp(self) :
       self.playerChannel.expressLost()
       self.playerChannel.expressAllCommands(self.commandVocab)
       self.playerChannel.expressAllDirections(self.dirVocab)



#___________________________________________________________________________________


# This class defines and manage a room. The universe of the game is made of interconnected rooms.
#Each room can have one or several exits. Through each of these exits, we arrive in another room.
#

class Room :

    #This is a constructor.
    #Creates a room described by "description". Initially, it has no exits.
    #"description" is something like "a kitchen" or "an open court yard".
    # parameter descriptionText is a String
    # parameter directions is a Vocabulary object
   def __init__(self, descriptionText, directions) :

       self.description = descriptionText
       self.possibleDirections = directions
       numberOfDirections = self.possibleDirections.getNumberOfWords(); # the method `getNumberOfWords()`
       # is defined in class `Vocabulary` possibleDirection is an object of this class `DirectionVocabulary`.
       
       self.connectedRooms = [] # this list will contain the rooms to which the curent room (self) is connected
       # For example, we could have :
       # connectedRoom[0] : room connected through the north exit if this exit exists
       # connectedRoom[1] : room connected through the east exit if this exit exists
       # connectedRoom[2] : room connected through the south exit if this exit exists
       # connectedRoom[3] : room connected through the west exit if this exit exists
       
       self.exits = [] # this list will contain booleans indicating  whether there is a connected room in this direction
       # For example, we could have:
       # exits[0] : north exit
       # exits[1] : east exit
       # exits[2] : south exit
       # exits[3] : west exit


       # initialization of the exit list : no exit
       i=0
       while i < numberOfDirections :
           self.exits.append(False)
           i = i + 1
       

       # initialization of connectedRooms
       # we start by putting None in all directions
       # None, in Python, is a sort of constant meaning "nothing" "empty"
       i=0
       while i < numberOfDirections :
           self.connectedRooms.append(None)
           i = i + 1


   
    #This method defines one exit of this room.
    #The direction is given by a text.
  
    #param direction - a word recognized as describing a direction.
    #param nextRoom  - an object Room, which will be reached if we follow this direction
    #
   def setExit(self, direction, nextRoom) :

       dirNum = self.possibleDirections.wordToCode(direction);
       # wordToCode(...) is defined in class `Vocabulary`. It translates a word of the lexicon in its
       # digital code (0, 1, 2, etc.).
       #possibleDirection is an object of the class `Vocabulary`.

       self.exits[dirNum] = True;
       self.connectedRooms[dirNum] = nextRoom;

   
    #Return the description of the room (the one that was defined in the constructor).
    
   def getShortDescription(self) :

       return self.description;
   


    #Return a long description of this room, on the form:
    #You are in the kitchen.
    #Exits: north west
    #
   def getLongDescription(self) :

       return "You are in " + self.description + ".\n" + self.exitString()
       # + allows to concatenate Strings (texts)
       # "\n" will be interpreted as Go to the line (return)
   

   
    #Returns whether there is an exit in the given direction
    # param direction is a word
    #If the direction does not exist, returns false.
    #
   def isExit(self, direction) :

       directionCode = self.possibleDirections.wordToCode(direction)
       if self.possibleDirections.isErrorCode(directionCode) :
           result = false;
       else :
           result = self.exits[directionCode]
       
       return result;
   
   
    #Returns the room (an object of class Room) that is reached if we go from this room in direction
    #"direction" (a word). There must be a room in this direction.
    #
   def nextRoom(self, direction) :

       directionCode = self.possibleDirections.wordToCode(direction)
       return self.connectedRooms[directionCode]

   
    #param otherRoom is a Room object
    #returns a boolean : true if this room is the same as the one proposed as a parameter
    #This method is necessary because we cannot use a simple ==.
    # Note that the comparison is made based on the description of the room
    #
   def isSameRoomAs(self, otherRoom) :

       return self.description == otherRoom.getShortDescription()


    #Returns a String describing the room's exits (separated by comas ans spaces), for example
    #"Exits: north, south ".
    #
   def exitString(self) :

       returnString = "Exits:"
       i=0
       while i < self.possibleDirections.getNumberOfWords() :
           if (self.exits[i]) :
               returnString = returnString + ", " + self.connectedRooms[i].getShortDescription()
           i=i+1
 
       return returnString



#________________________________________________________________________________


# This class manages the inputs tuped in by a player of the game.
# It can get a line entered thanks to the keyboard.
# It can analyze this line to separate individual words.
# It can give access to information about this line, about these words and access to each word.

class Parser :

    # This is the constructor of this class.
    #
    # param sentenceMaxLength - an integer defining the maximum number of words we can expect in one sentence.
  
   def __init__(self, sentenceMaxLength) :

       self.maxNbWords = sentenceMaxLength
       self.words = []
       self.numberOfWords = 0

    # returns a String (text) : the sentence typed in by a player.
    
   def getSentence(self) :
       print("> ") # print prompt
       return input(); # returns a `String`
                      # This `String` contains all what has been typed in, until the next `return`.


    # param sentence is a `String`, which what has been typed in by a player.
    # This method identifies the different words in this sentence.
    # This method stores in the attribute `words[]` each word at its real position.
    #
    # Example:
    #  - words[1] stores the 1st word of the sentence
    #  - words[2] stores ths second one...
    #
   def analyzeSentence(self, sentence) :
       theWordsOfTheSentence = sentence.split(" ") #this returns the list of all the words separated by a space (in sentence)
       self.words= ["-"]+theWordsOfTheSentence       # the list words contains "-" in position 0 and then every word
       self.numberOfWords = len(theWordsOfTheSentence)


    # This is a combo: it gets a sentence from the keyboard and stores each of its words in `words[]`.
    
   def getAndAnalyzeSentence(self) :
       sentence = self.getSentence();
       self.analyzeSentence(sentence);



    # param position (position of the word in the sentence. This starts at 1.
    #                 position is not checked. It should be between 1 and maxNbWords.
    # returns the corresponding word in the last analyzed sentence.
    
   def wordAt(self, position) :
       return self.words[position];


    # returns the number of words of the last sentence which has been analyzed with `analyzeSentence(String s)`
    
   def lastSentenceLength(self) :
       return self.numberOfWords


#_________________________________________________________________________________

class Vocabulary :

   
    # Constructor.
    # This method will be called when an object of this class will be created
    
   def __init__(self, words) :
       self.numberOfWords = len(words);
       self.wordList = words
       self.errorWord = "Error"
       self.errorCode = -1

   
    # param code is an int
    # returns the word ( as a String - ie text) corresponding to the code provided by the parameter (an integer).
    # If the code does not correspond to a word of this lexicon, the error word is returned
    
   def codeToWord(self, code) : 

       if (code >= self.numberOfWords) :
           result = self.errorWord
       else :
           result = self.wordList[code]
       
       return result
   

   
    # param word (a String - ie text)
    # returns the code (an int) corresponding to the word provided by the parameter
    # if this word does not exist in this lexicon, the error code is returned
    
   def wordToCode(self, word) :
       i = 0
       while i < self.numberOfWords and self.wordList[i]!=word :
           i = i + 1

       if i == self.numberOfWords :
           return self.errorCode; #we did not find the word
       else :
           return i


    # param word (a String - ie text)
    # return boolean (true or false) - returns true if the word provided by the parameter exists in this lexicon, false otherwise
    
   def isLexiconWord(self,  word) :
       
       i = 0
       while i < self.numberOfWords and self.wordList[i] != word :
           i = i + 1
       
       return (i < self.numberOfWords)
       # i<numberOfWords then the word has been found, otherwise is does not exist in this lexicon
   

   
    # This method is a getter
    #
    # returns the number of words in this lexicon
    # this is the only way to access this protected attribute from other classes than this one and
    # its derived classes
    
   def getNumberOfWords(self) :
       return self.numberOfWords
   

    # param code (an integer)
    # returns a boolean (true or false) - returns true if the code provided by the parameter is the
    #error code.
    # Thanks to this method, users of this class do not have to know what is the error code.
    # They will use this function to check whether the code at hand is an error code or not.
    
   def isErrorCode(self, code) :
       return (code == self.errorCode)
   

    # param word (a String - ie text)
    # return a boolean (true or false) - returns true if the word provided by the parameter is the error word.
    # Thanks to this method, users of this class do not have to know what is the error word.
    # They will use this function to check whether the word at hand is an error word or not.
    
   def isErrorWord(self, word) :
       return word==self.errorWord


    # returns a String (ie text) made of all the words in this lexicon, separated by a space (" ")
    
   def completeWordList(self) :
       zeList = ""
       for word in self.wordList :
         zeList = zeList + " " + word
       return zeList;
   

    # This method is a getter
    #
    # returns the errorCode (an integer)
    # this is the only way to access this private attribute
    
   def getErrorCode(self) :
       return self.errorCode

    # This method is a getter
    #
    # returns the errorWord (a String - ie text)
    # this is the only way to access this private attribute
    
   def getErrorWord()  :
       return self.errorWord

#_____________________________________________________________________________

class Interaction :


   def __init__(self) : # This is the constructor of the class
      self.messages = []
      self.messages.append("Sorry... but I do not understand what you mean.") # message 0
      self.messages.append("Bravo!") # message 1
      self.messages.append("What an extraordinary player you are!") # message 2
      self.messages.append("Thank you for playing.  Good bye.") # message 3
      self.messages.append("This adventure game is incredibly vintage!") # message 4
      self.messages.append("Type 'Help' if you need help.") # message 5
      self.messages.append("you have to reach ") # message 6
      self.messages.append("Hum... you need to choose a direction if you want to move!") # message 7
      self.messages.append("Sorry, there is no exit in this direction") # message 8
      self.messages.append("Sorry, I do not understand which direction you chose...") # message 9
      self.messages.append("Welcome!") # message 10
      self.messages.append("You are lost. You are alone. You wander around at ESCP Europe, Paris campus.") # message 11
      self.messages.append("Your command words are:") # message 12
      self.messages.append("The possible directions are:") # message 13

      self.commands = ["Go", "Quit", "Help"]
      self.directions = ["North", "East", "South", "West"]

   def getCommands(self):
     return self.commands

   def getDirections(self):
     return self.directions

   def expressGeneralMisunderstanding(self) :
       print(self.messages[0])

   def expressDirectionMisunderstanding(self):
       print(self.messages[9])

   def expressMissingDirection(self) :
       print(self.messages[7])

   def expressCongratulations(self) :
       print(self.messages[1])
       print(self.messages[2])

   def expressGoodBye(self) :
       print(self.messages[3])

   def expressVintage(self) :
       print(self.messages[4])

   def expressHowToGetHelp(self) :
       print(self.messages[5])

   def expressAllCommands(self, lex) : #we expect lex to be a Vocabulary
       print(self.messages[12])
       print(lex.completeWordList())

   def expressAllDirections(self, lex) :  #we expect lex to be a Vocabulary
       print(self.messages[13]);
       print(lex.completeWordList())

   def expressLost(self) :
       print(self.messages[11])

   def expressGoal(self,roomToReach) :  #we expect roomToReach to be an object of the classe Room
       print(self.messages[6] + roomToReach.getShortDescription())

   def expressInTheWall(self) :
       print(self.messages[8])

   def expressWelcome(self) :
       print(self.messages[10])

   def expressCurrentLocation(self, theRoom) : #we expect theRoom to be an object of the classe Room
       print(theRoom.getLongDescription())



#_____________________________________________________________________
# Main sequence
#_____________________________________________________________________

theGame = Game()
theGame.play()